In [3]:
# using ChromeDriver 89
from selenium import webdriver

PATH = './ChromeDriver/chromedriver'
driver = webdriver.Chrome(PATH)

In [4]:
# opening a website
driver.get("https://tutti.ch")

In [5]:
# close the browser
driver.quit()